In [1]:
import numpy as np
from copy import copy
import tensorflow as tf
from importlib import reload
from random import randint, sample, shuffle
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
size = plt.rcParams["figure.figsize"]
size[0] = 2
size[1] = 2
plt.rcParams["figure.figsize"] = size

import environment as env
import visual
import trainer as train
import agents.random_plus as rp
import agents.Q as Q
import agents.human as human
import agents.activated as activated
import util

In [69]:
_ = reload(env)
_ = reload(Q)
_ = reload(visual)
_ = reload(activated)
_ = reload(util)
_ = reload(train)

In [52]:
# Create an agent
agent = Q.Q([16, 100, 100, 16], gamma = .3, epsilon = 0.1, beta = 3.0, name = "4x4")
trainer = train.Trainer(agent, 1e-8)
# Create a random playing agent
RandomAgent = rp.RandomAgentPlus()
# Create Human agents (separate to keep memories separate)
HumanP1 = human.Human()
HumanP2 = human.Human()
# Create an environment with a 4x4 board
e = env.Env(4)

In [74]:
# Train against a random agent in normal mode
e.play(RandomAgent, agent, 2000, trainer_a2 = trainer.get_episode(1e-7, rotate = True),
       final_reward = True)

Playing ********** Done


In [62]:
# Multi-Purpose
gui = visual.GameWithConfidences(e, HumanP1, agent, -1, piece_size = 100, episode_train = True, final_reward = True,
                                 trainer_a1 = trainer.get_episode(1e-10))

In [14]:
agent.save()

In [49]:
(16*100 + 100*100 + 100*16 + 100 + 100 + 16) / (2 ** 16)

0.2047119140625